In [1]:
import cv2
from math import sqrt
import numpy as np 
from matplotlib import pyplot as plt
import sys, string, os

In [2]:
height = 500
width = 500

imgBlank = np.zeros((height,width, 3), np.uint)
url = "http://100.73.89.185:8080/video"
cap = cv2.VideoCapture(url)

In [3]:
def getContours(img, original_img):
    contours,hierachy = cv2.findContours(img, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area>60000:
            cv2.drawContours(original_img,cnt,-1,(0,255,0),3)
    
def biggestContour(img,original_img):
    contours,hierachy = cv2.findContours(img, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    biggest = np.array([])
    max_area = 0
    for cnt in contours:
        area= cv2.contourArea(cnt)
        if area>60000:
            cv2.drawContours(original_img,cnt,-1,(0,255,0),3)
            peri = cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt, 0.02*peri,True)
            if area>max_area and len(approx)==4:
                max_area = area
                biggest = approx
    cv2.imshow("Grid", original_img)
    return biggest, max_area

def reorder(points):
    points = points.reshape((4,2))
    pointsnew = np.zeros((4,1,2),dtype= np.int32)
    add = points.sum(1)
    pointsnew[0] = points[np.argmin(add)]
    pointsnew[3] = points[np.argmax(add)]
    diff = np.diff(points,axis=1)
    pointsnew[1] = points[np.argmin(diff)]
    pointsnew[2] = points[np.argmax(diff)]
    return pointsnew

def splitBoxes(img,n):
    rows = np.vsplit(img,n)
    boxes=[]
    for r in rows:
        cols = np.hsplit(r,n)
        for box in cols:
            boxes.append(box)
    global box_dimension 
    box_dimension = boxes[0].shape
    print(box_dimension)
    return boxes

col_val = [[0,0,254,1],[0,141,2,2],[4,224,234,3],[254,41,14,4],[0,137,250,5]]

def predict(boxes):
    board = []
    for image in boxes:
        col = (image[50,50])
        e = 0
        print(col)
        if col[0]<105 and col[1]<105 and col[2]<105:
            e=0
        elif col[0]<70 and col[1]>110 and col[1]<180 and col[2]>220:
            e = 1
        elif col[0]>220 and col[1]>175 and col[2]<100:
            e=2
        elif col[0]<70 and col[1]>215 and col[2]>125:
            e=3
        elif col[0]>70 and col[1]>230 and col[2]>230:
            e=4
        # elif col[0]<70 and col[1]>140 and col[1]<170 and col[2]>200:
        #     e=5
        board.append(e)
    return board

def display_output(img, output,n):
    a = b = 0
    k = l = 100
    m=0
    for i in range(0,n):
        for j in range(0,n):
            cv2.rectangle(img, pt1=(a,b),pt2=(k,l),color=col_val[int(output[m])-1],thickness=-1)
            a +=100
            k += 100
            m+=1
        a = 0
        k = 100
        b+=100
        l+=100
    cv2.imshow("Final", img)

In [4]:
while True:
    camera, img = cap.read()
    img = cv2.resize(img,(500,500))
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray,(3,3),0)
    imgCanny = cv2.Canny(imgBlur,50,200)
    img_copy = img.copy()

    biggest, maxArea = biggestContour(imgCanny,img_copy)

    if biggest.size!= 0:
        biggest = reorder(biggest)
        cv2.drawContours(img_copy, biggest, -1,(0,255,0),10)
        pts1 = np.float32(biggest)
        pts2 = np.float32([[0,0],[width,0],[0,height],[width,height]])
        matrix = cv2.getPerspectiveTransform(pts1,pts2)
        imgWarpColored = cv2.warpPerspective(img, matrix, (width,height))
        imgDetectedCircles = imgBlank.copy()
        # imgWarpColored = cv2.cvtColor(imgWarpColored,cv2.COLOR_BGR2GRAY)
    
    if maxArea!=0:
        cv2.imwrite('grid.jpg',img)
        break

    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) & 0xff == ord('q'):
        cv2.destroyAllWindows()
        break

print("hello")
if biggest.size!= 0:
    biggest = reorder(biggest)
    cv2.drawContours(img_copy, biggest, -1,(0,255,0),10)
    pts1 = np.float32(biggest)
    pts2 = np.float32([[0,0],[width,0],[0,height],[width,height]])
    matrix = cv2.getPerspectiveTransform(pts1,pts2)
    imgWarpColored = cv2.warpPerspective(img, matrix, (width,height))
    imgDetectedCircles = imgBlank.copy()
    # imgWarpColored = cv2.cvtColor(imgWarpColored,cv2.COLOR_BGR2GRAY)

imgSolvedDigits = imgBlank.copy()
n = 5
cv2.imshow("Warped", imgWarpColored)
cv2.imwrite('warped_grid.jpg',imgWarpColored)
# boxes = splitBoxes(imgWarpColored,n)
# board = predict(boxes)
# bt  = open('board.txt','w')
# bt.write(str(n)+"\n")
# for i in range(len(board)):
#     bt.write(str(board[i])+" ")
# bt.close()

# os.system("./a.out")

# ot = open('output.txt','r')
# data = ot.read()
# output = data.split(" ")
# print(output)
# display_output(imgWarpColored,output,n)

cv2.waitKey(0)
cv2.destroyAllWindows()

[mjpeg @ 0x36abd80] overread 8


hello
(100, 100, 3)
[ 44 151 255]
[192 196 183]
[115 111  86]
[92 55 40]
[ 38 110 251]
[61 42 49]
[254 245  35]
[ 59 229 141]
[252 211  18]
[170 255 254]
[86 26 16]
[110  41  19]
[85 47 20]
[100  42  22]
[37 54 63]
[21 12 14]
[ 38 236 141]
[27 18 16]
[33 34 30]
[40 53 67]
[17 15 19]
[31 32 33]
[34 41 44]
[221 255 254]
[50 68 78]
['Answer', 'not', 'possible\n']


ValueError: invalid literal for int() with base 10: 'Answer'